# Analysing steps in a short walk using acceleration and rotation

In this noteboook we examine a recording from the Arduino IMU of a short walk with the aim to extract the step count, cadence and timings including ground time and step duration.

We implement three techniques outlined in the paper `A comprehensive comparison of simple step counting techniques using wrist- and ankle-mounted accelerometer and gyroscope signals` by Matthew Rudy and Joseph Mahoney - [https://www.researchgate.net/publication/325451208_A_comprehensive_comparison_of_simple_step_counting_techniques_using_wrist-_and_ankle-mounted_accelerometer_and_gyroscope_signals](https://www.researchgate.net/publication/325451208_A_comprehensive_comparison_of_simple_step_counting_techniques_using_wrist-_and_ankle-mounted_accelerometer_and_gyroscope_signals).

* Peak-finding
* Fast Fourier Transform (FFT)
* Autocorrelation

Each of these methods allows us to count steps. The peak-finding method also identifies where the steps occur in the timeseries, so this in turn allows us to isolate steps and calculate such things as ground time and step duration.

## The IMU

The Arduino Nano inertial measurement unit gives us acceleration, measured in `g`s (`1g = 9.8m/s/s`), and rotation (angular velocity). In this notebook we demonstrate how to extract steps from either acceleration or rotation.

## The data

We expect a CSV file with columns for time, 3 axes of acceleration, and 3 axes of gyroscopic rotation.

## Setup

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.integrate import cumtrapz
from scipy.signal import butter, filtfilt, periodogram, spectrogram, find_peaks
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11, 4)})
import numpy as np
import gpxpy
from xml.etree import ElementTree as ET
from datetime import timedelta
from tqdm import tqdm_notebook
tqdm_notebook()

## Data import

In [ ]:
data_file_path = "../data/WearableMyFoot backgarden 1km - side orientation.csv"
# data_file_path = "../data/WearableMyFoot 0.3km backgarden run (device on foot).csv"
output_file_path = "../data/WearableMyFoot backgarden 1km - side orientation.gpx"
df = pd.read_csv(data_file_path)
df.columns = ["time", "aX", "aY", "aZ", "gX", "gY", "gZ"]
df.time = df.time - df.time.min()
df

## Key parameters

In [ ]:
# params
fs = 1000 / 10 # Hz, sampling frequency
timestamp = pd.to_datetime("2020-09-10T19:01:07")
total_time = df.time.max() - df.time.min()

## Helper functions

In [ ]:
def get_time_period(a, b):
    """
    a, b -- time in seconds
    """
    return df.loc[(df.time >= a * 1000) & (df.time < b * 1000)]

def band_pass(data, fc_low, fc_high, fs):
    w_low = fc_low / (fs / 2) # Normalize the frequency
    w_high = fc_high / (fs / 2) # Normalize the frequency
    b, a = butter(5, [w_low, w_high], 'bandpass')
    return filtfilt(b, a, data)

def high_pass(data, fc_low, fs):
    w_low = fc_low / (fs / 2) # Normalize the frequency
    b, a = butter(5, w_low, 'highpass')
    return filtfilt(b, a, data)

def low_pass(data, fc_high, fs):
    w_high = fc_high / (fs / 2) # Normalize the frequency
    b, a = butter(5, w_high, 'lowpass')
    return filtfilt(b, a, data)

def get_magnitude(data, fc_low=None, fc_high=None, fs=None):
    magnitude = np.sqrt((data**2).sum(axis=1))
    if fc_low is not None and fc_high is not None:
        return band_pass(magnitude, fc_low, fc_high, fs)
    elif fc_low is not None:
        return high_pass(magnitude, fc_low, fs)
    elif fc_high is not None:
        return low_pass(magnitude, fc_high, fs)
    else:
        return magnitude
    
def peak_detection_steps(data, pos_kwargs=None, neg_kwargs=None, plot=False):
    peaks, _ = find_peaks(data, **pos_kwargs)
    neg_peaks, _ = find_peaks(-data, **neg_kwargs)
    if plot:
        sns.lineplot(x=range(len(data)), y=data)
        sns.scatterplot(x=peaks, y=data[peaks])
        sns.scatterplot(x=neg_peaks, y=data[neg_peaks])
        plt.show()
    return max(len(peaks), len(neg_peaks))

def fft_dominant_freq(data, fs, plot=False):
    f, Pxx = periodogram(data, fs=fs)
    if plot:
        sns.lineplot(f, Pxx)
        plt.xlim([0.0,10.0])
        plt.show()
    return f[np.argmax(Pxx[10:])]

def fft_steps(data, dt, fs, plot=False):
    return dt * fft_dominant_freq(data, fs, plot) / 1000

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[:int(len(result)/2)]

def autocorr_steps(data, plot=False):
    corr = autocorr(data)
    peaks, _ = find_peaks(corr)
    if plot:
        sns.lineplot(x=range(len(corr)), y=corr)
        sns.scatterplot(x=peaks, y=corr[peaks])
        plt.show()
    return len(peaks)

def to_steps_per_minute(step_count, dt):
    """
    dt -- time in seconds
    """
    return step_count / dt * 60

def get_spm_for_period(a, b, columns, fc_low=None, fc_high=None, peak_detection_kwargs={}):
    frame = get_time_period(a, b)
    dt = (frame.time.max() - frame.time.min()) / 1000.0
    aMagnitude = get_magnitude(frame.loc[:,columns], fc_low=fc_low, fc_high=fc_high, fs=fs)
    n_peak_steps = peak_detection_steps(
        aMagnitude, 
        **peak_detection_kwargs
    )
    dominant_freq = fft_dominant_freq(aMagnitude, fs=fs)
    n_autocorr_steps = autocorr_steps(aMagnitude)
    return pd.Series({
        "peak_detection_spm": to_steps_per_minute(n_peak_steps, dt), 
        "fft_spm": dominant_freq * 60, 
        "autocorrelation_spm": to_steps_per_minute(n_autocorr_steps, dt), 
    })

def get_spm(data, a, b, fc_low=None, fc_high=None, peak_detection_kwargs={}, peak_detection=True, fft=True, autocorrelation=True):
    frame = get_time_period(a, b)
    dt = (frame.time.max() - frame.time.min()) / 1000.0
    _data = data.loc[frame.index].values
    if fc_low is not None:
        _data = high_pass(_data, fc_low, fs)
    if fc_high is not None:
        _data = low_pass(_data, fc_high, fs)
        
#     sns.lineplot(x=range(len(_data)), y=_data)
    n_peak_steps = peak_detection_steps(
        _data, 
        **peak_detection_kwargs
    ) if peak_detection else 0.0
    dominant_freq = fft_dominant_freq(_data, fs=fs) if fft else 0.0
    n_autocorr_steps = autocorr_steps(_data) if autocorrelation else 0.0
    return pd.Series({
        "peak_detection_spm": to_steps_per_minute(n_peak_steps, dt), 
        "fft_spm": dominant_freq * 60, 
        "autocorrelation_spm": to_steps_per_minute(n_autocorr_steps, dt), 
    })

def scalar_projection(u, v):
    """
    project u on v
    """
    v_norm = np.sqrt(sum(v**2))
    return (np.dot(u, v)/v_norm)



# Measuring steps from acceleration

In [ ]:
sns.lineplot(x=df.time, y=df.aX, label="aX")
sns.lineplot(x=df.time, y=df.aY, label="aY")
sns.lineplot(x=df.time, y=df.aZ, label="aZ")

In [ ]:
_df = df.iloc[4125:4200].loc[:, ["aX", "aY", "aZ"]]
_df.plot()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the values
ax.scatter(_df.aX, _df.aY, _df.aZ, c = 'b', marker='o')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

plt.show()

In [ ]:
def get_pca(data):
    pca = PCA()

    pca.fit(data)
    print(pca.explained_variance_ratio_)

    return pd.Series(np.reshape(pca.components_, (9,)), index=["x1", "y1", "z1", "x2", "y2", "z2", "x3", "y3", "z3"])

step = 75

components = pd.DataFrame([get_pca(df.iloc[a:a+step].loc[:, ["aX", "aY", "aZ"]]) for a in np.arange(0,df.shape[0],step)])
components.loc[:, ["x1", "y1", "z1"]].plot()

In [ ]:
get_pca(df.loc[:, ["aX", "aY", "aZ"]])

In [ ]:
_df = df.iloc[:9]
sns.lineplot(x=_df.time, y=_df.aX, label="aX")
sns.lineplot(x=_df.time, y=_df.aY, label="aY")
sns.lineplot(x=_df.time, y=_df.aZ, label="aZ")

In [ ]:
gravity_vector = _df.loc[:,["aX", "aY", "aZ"]].mean()
(gravity_vector, (gravity_vector**2).sum())

In [ ]:
df.loc[:,["aX", "aY", "aZ"]] = (df.loc[:,["aX", "aY", "aZ"]] - gravity_vector) * 9.8

In [ ]:
df.loc[:,["pca0", "pca1", "pca2"]] = PCA().fit_transform(df.loc[:,["aX", "aY", "aZ"]])

In [ ]:
vertical = df.loc[:,["aX", "aY", "aZ"]].apply(scalar_projection, v=gravity_vector, axis=1)

In [ ]:
# f, t, Sxx = spectrogram(get_magnitude(df.loc[:,["aX", "aY", "aZ"]], fs=fs), fs)
f, t, Sxx = spectrogram(df.pca2, fs)
# f, t, Sxx = spectrogram(vertical, fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim([0,10])
plt.show()

In [ ]:
# aMagnitude = get_magnitude(df.loc[:,["aX", "aY", "aZ"]], fc_low=0.3, fc_high=None, fs=fs)
aMagnitude = high_pass(df.pca0, fc_low=0.3, fs=fs)
# aMagnitude = low_pass(vertical, fc_high=1.5, fs=fs)
# aMagnitude = high_pass(aMagnitude, fc_low=1.0, fs=fs)
# aMagnitude = band_pass(vertical, fc_low=0.5, fc_high=2.0, fs=fs)
# aMagnitude = vertical[4000:4500]
n_peak_steps = peak_detection_steps(
    aMagnitude, 
    pos_kwargs={
        "prominence": 2,
        "distance": 20,
        "height": (35, None)
    }, 
    neg_kwargs={
        "prominence": 2,
        "distance": 20,
        "height": (35, None)
    },
    plot=True
)
n_fft_steps = fft_steps(aMagnitude, dt=total_time, fs=fs, plot=True)
n_autocorr_steps = autocorr_steps(aMagnitude, plot=True)
print(n_peak_steps, n_fft_steps, n_autocorr_steps)
print(
    to_steps_per_minute(n_peak_steps, total_time / 1000), 
    to_steps_per_minute(n_fft_steps, total_time / 1000), 
    to_steps_per_minute(n_autocorr_steps, total_time / 1000), 
)

In [ ]:
a = 25000
peak_detection_steps(
    df.pca0[a:a+500].values, 
    pos_kwargs={
#         "prominence": 20,
        "distance": 20,
        "height": (35, None)
    }, 
    neg_kwargs={
        "distance": 20,
        "height": (35, None)
    },
    plot=True
)
autocorr_steps(df.pca0[a:a+500].values, plot=True)

In [ ]:
peak_detection_kwargs = {
    "pos_kwargs": {
        "prominence": 20,
        "distance": 20,
        "height": (35, None)
    }, 
    "neg_kwargs": {
        "prominence": 20,
        "distance": 20,
        "height": (35, None)
    },
}

spms = []
dt = 10
step = 3.0
for a in tqdm_notebook(np.arange(0, df.time.max() / 1000, step)):
    b = a + dt
    spm = get_spm(df.pca0, a, b, fc_low=None, fc_high=None, peak_detection_kwargs=peak_detection_kwargs, autocorrelation=False, fft=False)
#     spm = get_spm(df.pca0, a, b, fc_low=None, fc_high=4.0, peak_detection_kwargs=peak_detection_kwargs, autocorrelation=True, fft=False)
    spm.name = a
    spms += [spm]
#     break
spm_df = pd.DataFrame(spms)
spm_df.plot()
spm_df.describe()

# Measuring steps from rotation

In [ ]:
sns.lineplot(x=df.time, y=df.gX, label="gX")
sns.lineplot(x=df.time, y=df.gY, label="gY")
sns.lineplot(x=df.time, y=df.gZ, label="gZ")

In [ ]:
f, t, Sxx = spectrogram(get_magnitude(df.loc[:,["gX", "gY", "gZ"]], fs=fs), fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
gMagnitude = get_magnitude(df.loc[:,["gX", "gY", "gZ"]], fc_low=0.5, fc_high=3.0, fs=fs)
n_peak_steps = peak_detection_steps(
    gMagnitude, 
    pos_kwargs={
        "prominence": 2,
        "distance": 3
    }, 
    neg_kwargs={
        "prominence": 1
    },
    plot=True
)
n_fft_steps = fft_steps(gMagnitude, dt=1000, fs=fs, plot=True)
n_autocorr_steps = autocorr_steps(gMagnitude, plot=True)
print(n_peak_steps, n_fft_steps, n_autocorr_steps)
print(
    to_steps_per_minute(n_peak_steps, total_time / 1000), 
    to_steps_per_minute(n_fft_steps, total_time / 1000), 
    to_steps_per_minute(n_autocorr_steps, total_time / 1000), 
)

In [ ]:
peak_detection_kwargs = {
    "pos_kwargs": {
        "height": (50, None)
    }, 
    "neg_kwargs": {
        "prominence": 1
    },
}

spms = []
dt = 3
for a in np.arange(0, df.time.max() / 1000, dt):
    b = a + dt
    spm = get_spm_for_period(a, b, ["gX", "gY", "gZ"], fc_low=0.5, fc_high=2.0, peak_detection_kwargs=peak_detection_kwargs)
    spm.name = a
    spms += [spm]
spm_df = pd.DataFrame(spms)
spm_df.plot()
spm_df.describe()


# Export GPX
Contains a track with cadence

In [ ]:
def get_cadence_extension(cadence):
    prefix = "gpxtrx:"
#     prefix = "{http://www.garmin.com/xmlschemas/TrackPointExtension/v1}"
    element = ET.Element(f"{prefix}TrackPointExtension")
    cadence_element = ET.SubElement(element, f"{prefix}cad")
    # Schema only permits integers up to 254
    cadence_element.text = str(int(cadence if cadence <= 254 else 254))
    return element

def get_point(time, cadence):
    extensions = [get_cadence_extension(cadence)]
    point = gpxpy.gpx.GPXTrackPoint()
    point.extensions = extensions
    point.time = time
    return point

def get_gpx(data):
    """
    data -- pandas DataFrame with time and cadence fields
    """
    gpx = gpxpy.gpx.GPX()
    gpx.nsmap["gpxtrx"] = 'http://www.garmin.com/xmlschemas/GpxExtensions/v3'
    track = gpxpy.gpx.GPXTrack()
    gpx.tracks.append(track)
    segment = gpxpy.gpx.GPXTrackSegment()
    track.segments.append(segment)
    segment.points = [get_point(x.time, x.peak_detection_spm) for _, x in data.iterrows()]
    return gpx
    
spm_df.loc[:,"time"] = [timestamp + timedelta(seconds=x) for x in spm_df.index]
with open(output_file_path, 'w+') as f:
    f.write(get_gpx(spm_df).to_xml())